In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
torch.manual_seed(123)

In [ ]:
def scaled_dot_product(query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: torch.Tensor):
    
        embedding_dimension = torch.tensor(
        query.size()[-1],
        dtype=torch.float32
        )

        # The nominator of the self attention
        attention_score: torch.Tensor = torch.matmul(
                query,
                key.transpose(-2, -1)
                )
        
        attention_weights: torch.Tensor = attention_score / torch.sqrt(embedding_dimension)
        
        if mask is not None:
                attention_weights.masked_fill_(mask.bool(), -torch.inf)

        attention_weights = F.softmax(attention_weights, dim=-1)
        output = torch.matmul(attention_weights, value)
        return output
        

In [3]:
x: torch.Tensor = torch.tensor(
    [[0.43, 0.15, 0.89],    
     [0.55, 0.87, 0.66],    
     [0.57, 0.85, 0.64],    
     [0.22, 0.58, 0.33],    
     [0.77, 0.25, 0.10],    
     [0.05, 0.80, 0.55]]
     )

In [27]:
embedding_dim: int = x.shape[-1]
sequence_length: int = x.shape[-2]

In [39]:
query_w: nn.Linear = nn.Linear(embedding_dim, embedding_dim, bias=False)
queries = query_w(x)

key_w: nn.Linear = nn.Linear(embedding_dim, embedding_dim, bias=False)
keys: torch.Tensor = key_w(x)

value_w: nn.Linear = nn.Linear(embedding_dim, embedding_dim, bias=False)
values = value_w(x)

mask: torch.Tensor = torch.triu(torch.ones(sequence_length, sequence_length), diagonal=1)


In [40]:
context_vector: torch.Tensor = scaled_dot_product(query=queries, key=keys, value=values, mask=mask)